In [1]:
import win32com.client
import os

def add_and_run_macro(excel_file_path, macro_code):
    # Open Excel application
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False 

    # Open the workbook
    wb = excel.Workbooks.Open(excel_file_path)

    # Get the VBA project
    vb_project = wb.VBProject

    # Add a new module
    vb_component = vb_project.VBComponents.Add(1)  # 1 = vbext_ct_StdModule

    # Add the macro code to the module
    vb_component.CodeModule.AddFromString(macro_code)

    # Run the macro
    excel.Application.Run("cleaning")

    # Save and close
    wb.Save()
    wb.Close()
    excel.Quit()

# Your VBA macro code
macro_code = """
Sub cleaning()

Dim wb As Workbook
Dim ws As Worksheet
Dim crr_cell As Range
Dim rangeToFormat As Range
Dim proj_name As String
Set wb = ThisWorkbook
Set ws = wb.Sheets(1)

lastrow = ws.Cells(Rows.Count, 1).End(xlUp).Row
ws.Columns(1).Insert Shift:=xlToRight
ws.Cells.Select
Selection.UnMerge
ws.Rows("24:24").Copy
ws.Rows("18:18").PasteSpecial
ws.Cells(18, 1).Select

'Copying Project Names
For i = 18 To lastrow
    Set crr_cell = ws.Cells(i, 2)
    If crr_cell.Interior.Color = 11891758 And crr_cell.Value = "Bank Account Name" Then
        proj_name = crr_cell.Offset(-4, 3)
        ws.Cells(i, 1) = proj_name
    ElseIf crr_cell.Interior.Color = 16777215 And crr_cell.Value <> "" And crr_cell.Offset(0, 1).Value <> "" Then
        ws.Cells(i, 1) = proj_name
    End If
Next i

lastrow = ws.Cells(Rows.Count, 1).End(xlUp).Row
With ws.Cells(18, 1)
    .Value = "Project"
    .Interior.Color = 11891758
    .Font.Color = RGB(255, 255, 255) 'white color
End With
ws.Cells(lastrow + 1, 1).Value = "END"

'Deleting excess rows
c = 19
Do While ws.Cells(c, 1).Value <> "END"
    Set crr_cell = ws.Cells(c, 1)
    If crr_cell.Value = "" Or crr_cell.Offset(0, 1) = "" Or crr_cell.Offset(0, 1) = "Bank Account Name" Or crr_cell.Offset(0, 1) = "Owner Name " Then
        Rows(crr_cell.Row).Delete
    Else
        c = c + 1
    End If
Loop

lastrow = ws.Cells(Rows.Count, 1).End(xlUp).Row
ws.Rows(lastrow & ":" & (lastrow + 50)).Delete

lastrow = ws.Cells(Rows.Count, 1).End(xlUp).Row

Set rangeToFormat = ws.Range(ws.Cells(18, 1), ws.Cells(lastrow, 15))

With rangeToFormat
    .Borders.LineStyle = xlContinuous
    .Borders.ColorIndex = 0
    .Borders.TintAndShade = 0
    .Borders.Weight = xlThin
    .HorizontalAlignment = xlCenter
End With

ws.Cells.EntireColumn.AutoFit


End Sub
"""

# Get the directory where your Jupyter Notebook is located
notebook_directory = os.path.dirname(os.path.abspath("__file__")) 

# Find all Excel files in the directory
excel_files = [f for f in os.listdir(notebook_directory) if f.endswith('.xlsx') or f.endswith('.xls')]

# Process each Excel file
for excel_file in excel_files:
    excel_file_path = os.path.join(notebook_directory, excel_file)
    add_and_run_macro(excel_file_path, macro_code)
    print(f"Processed: {excel_file}")

Processed: 23.aug.only.RME_AP_Check_Payment_Grouping__130923.xlsx
